#### 1. Import pandas library

In [1]:
import pandas as pd 

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data 


In [2]:
import pymysql
from sqlalchemy import create_engine

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/search?tableCount%5B%5D=0-10&tableCount%5B%5D=10-30&dataType%5B%5D=Numeric&databaseSize%5B%5D=KB&databaseSize%5B%5D=MB)

In [3]:
"""
ip = "35.239.232.23"

login = "ironhacker_read"

password = "ir0nhack3r"

db = "sakila"

engine = create_engine(f'mysql+pymysql://{login}:{password}@{ip}/{db}')
"""


hostname = "relational.fit.cvut.cz"
port = "3306"
username = "guest"
password = "relational"


engine = create_engine(f"mysql+pymysql://{username}:{password}@{hostname}:{port}")

#### 4. Import the users table 

In [4]:
#data = pd.read_sql_query('SELECT * FROM publications.employee', engine)

users_table = pd.read_sql_query('SELECT * FROM stats.users', engine)

In [5]:
users_table.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None


#### 5. Rename Id column to userId

In [6]:
users_table.rename(columns= {"Id": "userId"}, inplace= True) 

In [7]:
users_table.head()

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None


In [8]:
users_table["userId"]

0           -1
1            2
2            3
3            4
4            5
5            6
6            7
7            8
8           10
9           11
10          12
11          13
12          15
13          16
14          17
15          18
16          19
17          21
18          22
19          23
20          24
21          25
22          26
23          27
24          29
25          30
26          33
27          34
28          35
29          36
         ...  
40295    55718
40296    55719
40297    55720
40298    55721
40299    55722
40300    55723
40301    55724
40302    55725
40303    55726
40304    55727
40305    55728
40306    55729
40307    55730
40308    55731
40309    55732
40310    55733
40311    55734
40312    55735
40313    55736
40314    55737
40315    55738
40316    55739
40317    55740
40318    55741
40319    55742
40320    55743
40321    55744
40322    55745
40323    55746
40324    55747
Name: userId, Length: 40325, dtype: int64

#### 6. Import the posts table. 

In [9]:
post_table = pd.read_sql_query("SELECT * FROM stats.posts", engine)

In [10]:
post_table.head()

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaT,NaT,NaN,NaT,None,None
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaT,NaN,NaT,None,None
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaT,None,None
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaT,NaT,NaN,NaT,None,None
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,None,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaT,None,None


#### 7. Rename Id column to postId and OwnerUserId to userId

In [11]:
post_table.rename( columns= {"Id": "postId", "OwnerUserId" : "userId"}, inplace = True)

#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [12]:
new_user_df = users_table[["userId", "Reputation", "Views", "UpVotes", "DownVotes"]]

In [13]:
new_user_df.head()

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5


In [14]:
new_user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40325 entries, 0 to 40324
Data columns (total 5 columns):
userId        40325 non-null int64
Reputation    40325 non-null int64
Views         40325 non-null int64
UpVotes       40325 non-null int64
DownVotes     40325 non-null int64
dtypes: int64(5)
memory usage: 1.5 MB


In [15]:
new_post_df = post_table[["postId", "Score", "userId", "ViewCount", "CommentCount"]]

In [16]:
new_post_df.head()

,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


In [17]:
new_post_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91976 entries, 0 to 91975
Data columns (total 5 columns):
postId          91976 non-null int64
Score           91976 non-null int64
userId          90584 non-null float64
ViewCount       42921 non-null float64
CommentCount    91976 non-null int64
dtypes: float64(2), int64(3)
memory usage: 3.5 MB


#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [18]:
merge_df = new_post_df.merge(new_user_df, how= "outer")

In [19]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110318 entries, 0 to 110317
Data columns (total 9 columns):
postId          91976 non-null float64
Score           91976 non-null float64
userId          108926 non-null float64
ViewCount       42921 non-null float64
CommentCount    91976 non-null float64
Reputation      108926 non-null float64
Views           108926 non-null float64
UpVotes         108926 non-null float64
DownVotes       108926 non-null float64
dtypes: float64(9)
memory usage: 8.4 MB


In [20]:
merge_df.head()

,postId,Score,userId,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes
0,1.0,23.0,8.0,1278.0,1.0,6764.0,1089.0,604.0,25.0
1,16.0,16.0,8.0,NaN,3.0,6764.0,1089.0,604.0,25.0
2,36.0,41.0,8.0,67396.0,7.0,6764.0,1089.0,604.0,25.0
3,65.0,14.0,8.0,NaN,3.0,6764.0,1089.0,604.0,25.0
4,78.0,33.0,8.0,NaN,4.0,6764.0,1089.0,604.0,25.0


#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [22]:
merge_df.isna().sum()

postId          18342
Score           18342
userId           1392
ViewCount       67397
CommentCount    18342
Reputation       1392
Views            1392
UpVotes          1392
DownVotes        1392
dtype: int64

#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

I'll fill them with 0, as they are values with no value.

In [28]:
merge_fill = merge_df.fillna(0)

In [29]:
merge_fill

,postId,Score,userId,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes
0,1.0,23.0,8.0,1278.0,1.0,6764.0,1089.0,604.0,25.0
1,16.0,16.0,8.0,0.0,3.0,6764.0,1089.0,604.0,25.0
2,36.0,41.0,8.0,67396.0,7.0,6764.0,1089.0,604.0,25.0
3,65.0,14.0,8.0,0.0,3.0,6764.0,1089.0,604.0,25.0
4,78.0,33.0,8.0,0.0,4.0,6764.0,1089.0,604.0,25.0
5,111.0,8.0,8.0,0.0,3.0,6764.0,1089.0,604.0,25.0
6,114.0,30.0,8.0,1220.0,1.0,6764.0,1089.0,604.0,25.0
7,129.0,4.0,8.0,0.0,0.0,6764.0,1089.0,604.0,25.0
8,142.0,5.0,8.0,0.0,0.0,6764.0,1089.0,604.0,25.0
9,168.0,17.0,8.0,1022.0,1.0,6764.0,1089.0,604.0,25.0


#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

The Id should be objects as they are immutable. The others should be integers because they have no decimals

In [30]:
merge_fill.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110318 entries, 0 to 110317
Data columns (total 9 columns):
postId          110318 non-null float64
Score           110318 non-null float64
userId          110318 non-null float64
ViewCount       110318 non-null float64
CommentCount    110318 non-null float64
Reputation      110318 non-null float64
Views           110318 non-null float64
UpVotes         110318 non-null float64
DownVotes       110318 non-null float64
dtypes: float64(9)
memory usage: 8.4 MB


In [33]:
merge_fill.columns

Index(['postId', 'Score', 'userId', 'ViewCount', 'CommentCount', 'Reputation',
       'Views', 'UpVotes', 'DownVotes'],
      dtype='object')

In [ ]:
merge_fill.astype(merge_fill[])

In [34]:
merge_fill = merge_fill.astype({"postId": object, "userId": object, 'Score': int, 'ViewCount':int, 'CommentCount':int, 'Reputation':int,
       'Views':int, 'UpVotes':int, 'DownVotes':int })

In [35]:
merge_fill.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110318 entries, 0 to 110317
Data columns (total 9 columns):
postId          110318 non-null object
Score           110318 non-null int64
userId          110318 non-null object
ViewCount       110318 non-null int64
CommentCount    110318 non-null int64
Reputation      110318 non-null int64
Views           110318 non-null int64
UpVotes         110318 non-null int64
DownVotes       110318 non-null int64
dtypes: int64(7), object(2)
memory usage: 8.4+ MB
